In [1]:
## Задание 1.
## *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - np.clip).
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 2, 1, 3, 0, 5, 10, 1, 2], # стаж
 [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000], # стоимость 
 [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64) # квалификация репет
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # подходит или нет репетитор
def calc_logloss(y, y_pred):
    min_y = 0.001
    max_y = 1
    y_pred = np.clip(y_pred, min_y, max_y)
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [19]:
## Задание 2.
## Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным.

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res  

def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        # np.clip, заменим нули на 0.00001
        y_pred = np.clip(y_pred, 0.00001, 0.99999)
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 50) == 0:
            print(i, w, err)
            return w
        
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [ ]:
## Решение

In [21]:
alphas_list = [1e-6, 8e-6, 9e-6, 1e-5]
for alpha in alphas_list:
    print(f'Alpha: {alpha}')
    eval_LR_model(X, y, 20000, alpha)
    print('-' * 100)

Alpha: 1e-06
400 [ 0.49651416 -0.13878429  0.46369244  1.52274986] 5.756467732512389
----------------------------------------------------------------------------------------------------
Alpha: 8e-06
400 [ 0.49586173 -0.13999359 -0.00242501  1.52212494] 0.60472809749708
----------------------------------------------------------------------------------------------------
Alpha: 9e-06
400 [ 0.4958568  -0.140096   -0.00360149  1.5221721 ] 0.7273651977701474
----------------------------------------------------------------------------------------------------
Alpha: 1e-05
400 [ 0.49585372 -0.14018245 -0.00414978  1.52222096] 0.8479099890773417
----------------------------------------------------------------------------------------------------


In [25]:
weights = eval_LR_model(X, y, 20000, 8e-6)

400 [ 0.49586173 -0.13999359 -0.00242501  1.52212494] 0.60472809749708


In [ ]:
## Alpha: 8e-06 показало себя лучше всего

In [26]:
## Задание 3.
## Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 
## (на вход подаются веса, которые уже посчитаны функцией eval_LR_model и X, на выходе - массив y_pred_proba).

def calc_pred_proba(w, X):
    return sigmoid(np.dot(w, X))
calc_pred_proba(weights, X)

array([0.66049219, 0.54499953, 0.8086663 , 0.60419789, 0.40224181,
       0.5194849 , 0.67365312, 0.23366728, 0.68713067, 0.69743538])

In [28]:
## Задание 4.
## Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются веса, которые уже посчитаны функцией
## eval_LR_model и X, на выходе - массив y_pred).
def calc_pred(w, X):
    threshold = 0.5
    y_pred_proba = sigmoid(np.dot(w, X))
    return (y_pred_proba > threshold).astype(np.int)
calc_pred_proba(weights, X)



array([0.66049219, 0.54499953, 0.8086663 , 0.60419789, 0.40224181,
       0.5194849 , 0.67365312, 0.23366728, 0.68713067, 0.69743538])

In [29]:
y_pred = calc_pred(weights, X)
y_pred

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [53]:
## Задание 5.
## Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

In [72]:
def confusion_matrix(y_true, y_pred):
    if len(y_true) != len(y_pred):
        raise ValueError('Размеры массивов неодинаковы.')
    conf_matrix = {
        'tp': 0,
        'tn': 0,
        'fp': 0,
        'fn': 0,
    }
    for i, _ in enumerate(y_true):
        if y_true[i] == 1:
            if y_pred[i] == y_true[i]:
                element = 'tp'
            else:
                element = 'fn'
        else:
            if y_pred[i] == y_true[i]:
                element = 'tn'
            else:
                element = 'fp'
        conf_matrix[element] += 1
    return conf_matrix

In [73]:
def show_confusion_matrix(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    print('\ttrue_1\ttrue_0')
    print(f'pred_1\t{conf_matrix["tp"]}\t{conf_matrix["fp"]}')
    print(f'pred_0\t{conf_matrix["fn"]}\t{conf_matrix["tn"]}')

In [74]:
def precision(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    return (conf_matrix['tp'] + conf_matrix['fp']) / len(y_true)

In [75]:
def recall(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    return (conf_matrix['tp'] + conf_matrix['fn']) / len(y_true)

In [76]:
def f1_score(y_true, y_pred):
    return 2 / (1 / precision(y_true, y_pred) + 1 / recall(y_true, y_pred))

In [77]:
accuracy(y, y_pred)

0.5

In [78]:
confusion_matrix(y, y_pred)

{'tp': 4, 'tn': 1, 'fp': 4, 'fn': 1}

In [79]:
show_confusion_matrix(y, y_pred)

	true_1	true_0
pred_1	4	4
pred_0	1	1


In [80]:
precision(y, y_pred)

0.8

In [82]:
recall(y, y_pred)

0.5

In [103]:
f1_score(y, y_pred)

0.6153846153846154

In [104]:
## Задание 6.
## Могла ли модель переобучиться? Почему?
## Ответ: МОГЛА. Модели всегда могут переобучиться, но в данном случае у модели не особо высокая точность,
## поэтому однозначно не могу сказать к ому же нет тестового датасета для сравнения.

In [119]:
## Задание 7.
## Создайте функции eval_LR_model_l1 и eval_LR_model_l2 с применением L1 и L2 регуляризации соответственно.
def eval_LR_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    m = X.shape[0]
    n = X.shape[1]
    for i in range(1, iterations + 1):
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        y_pred = np.clip(y_pred, 0.00001, 0.99999)
        err = calc_logloss(y, y_pred) + lambda_ / m * np.linalg.norm(w, ord=1)
        w -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ / m * np.sum(np.sign(w)))
        if i % (iterations / 50) == 0:
            print(i, w, err)
        return w   
eval_LR_model_l1(X, y, iterations=20000, alpha=8e-6, lambda_=5e-2)

array([ 0.49670995, -0.1382749 ,  0.64400842,  1.52302406])

In [120]:
def eval_LR_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    m = X.shape[0]
    n = X.shape[1]
    for i in range(1, iterations + 1):
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        y_pred = np.clip(y_pred, 0.00001, 0.99999)
        err = calc_logloss(y, y_pred) + lambda_ / (2 * m) * np.linalg.norm(w, ord=2)
        w -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ / m * np.sum(w))
        if i % (iterations / 50) == 0:
            print(i, w, err)
        return w
eval_LR_model_l2(X, y, iterations=20000, alpha=8e-6, lambda_=5e-2)

array([ 0.4967099 , -0.13827495,  0.64400836,  1.523024  ])